In [177]:
from pyspark import SparkConf, SparkContext
from re import split

# Create SparkContext object
conf = SparkConf().setMaster("local").setAppName("RatingsValueCounts")
sc = SparkContext(conf=conf)

# Create lines RDD from data file
data_file_path = "../data/fakefriends.csv"
lines = sc.textFile(data_file_path)
lines.take(2)

['0,Will,33,385', '1,Jean-Luc,26,2']

In [174]:
def parse_line(line: str):
    row = split(",", line)
    age = int(row[2])
    num_friends = int(row[3])
    return (age, num_friends)


age_friends = lines.map(parse_line)
age_friends.take(2)

[(33, 385), (26, 2)]

In [175]:
mean_friends_per_age = (
    age_friends.mapValues(lambda x: [x, 1])
    .reduceByKey(
        lambda x, y: [x[0] + y[0], x[1] + y[1]]
    )  # [[x1,1], [x2, 1], ...] -> [sum, count]
    .mapValues(lambda x: x[0] / x[1])  # [sum. count] -> mean
)
mean_friends_per_age.take(2)

[(33, 325.3333333333333), (26, 242.05882352941177)]

In [178]:
sc.stop()